We set our environment for easy access to settings

In [ ]:
%env THREADS 12

We assume conda is installed properly and the user has write access to create and run an environment.

We also assume enough available resources for the run. The major one is 64gb or memory. 

The rest of the parameter files assume a 12 thread CPU (the one we had for the project).

We setup an environment and apply some must needed fixes to all the applicable software:

In [ ]:
# !conda create -n l1em python=3.11 openjdk=21 -y
# !conda activate l1em
%conda config --add channels defaults
%conda config --prepend channels bioconda
%conda config --prepend channels conda-forge
# %conda install -y python=3.11 openjdk=21 bwa samtools bioconda/label/cf201901::flux-simulator numpy scipy sra-tools trim-galore pysam bedtools pigz wget
%conda install -y python=3.11 openjdk=21 bwa samtools flux-simulator numpy scipy sra-tools trim-galore pysam bedtools pigz wget ipython=8.14 bioconda::pygtftk

# Edit the flux-simulator caller script to overcome safety limitation due to old age coding standards (java code reflection)
!if ! grep -q -- '--add-opens java.base/java.util=ALL-UNNAMED' $CONDA_PREFIX/share/flux-simulator-1.2.1-3/bin/flux-simulator; then sed -i '/^java -Xmx\$FLUX_MEM.*/a --add-opens java.base/java.util=ALL-UNNAMED \\\n--add-opens java.desktop/java.awt.font=ALL-UNNAMED \\\n--add-opens java.base/java.text=ALL-UNNAMED' $CONDA_PREFIX/share/flux-simulator-1.2.1-3/bin/flux-simulator; fi

# Install beers2 and camparee (which is automaticall installed with beers2 and must not be installed separetely)
%pip install git+https://github.com/itmat/BEERS2

# Download the L1EM project from git
!git clone https://github.com/FenyoLab/L1EM


Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package pygtftk-1.0.1-py36hebb334c_0 requires python >=3.6,<3.7.0a0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ pygtftk is installable with the potential options
│  ├─ pygtftk [1.0.1|1.0.2|...|1.1.0] would require
│  │  └─ python >=3.6,<3.7.0a0 , which can be installed;
│  ├─ pygtftk 1.1.0 would require
│  │  └─ python >=3.7,<3.8.0a0 , which can be installed;
│  ├─ pygtftk [1.1.1|1.1.2|...|1.2.7] would require
│  │  ├─ python >=3.6,<3.7.0a0 , which can be installed;
│  │  └─ python_abi 3.6.* *_cp36m, which can be installed;
│  ├─ pygtftk [1.1.1|1

We go ahead and download the proper experiment data and the reference genomes, while also producing the relevant files from each download:

In [ ]:
%%bash
mkdir -p ./data/experiment
if [[ ! -f ./data/experiment/SRR3997504_1.fastq || ! -f ./data/experiment/SRR3997504_2.fastq ]]; then
    echo "One or both FASTQ files do not exist in the data folder. Running fasterq-dump..."
    fasterq-dump -O ./data/experiment SRR3997504
else
    echo "Both FASTQ files already exist in the data folder. Skipping fasterq-dump."
fi
if [[ ! -f ./data/experiment/SRR3997504_2_val_2_fastqc.html ]]; then
    "No trimmed files by trim_galore found. Running trim_galore..."
    trim_galore -j $((THREADS/4)) --paired --fastqc -o ./data/experiment/ data/experiment/SRR3997504_1.fastq ./data/experiment/SRR3997504_2.fastq
else
    echo "Trimmed files by trim_galore already exist. Skipping trim_galore."
fi


#https://zenodo.org/records/5146236/files/star.tar.gz?download=1


mkdir -p ./data/genome
if [[ ! -f ./data/genome/hg38.fa ]]; then
    echo "Humane genome 38 does not exist in the data folder. Downloading, unziping and indexing it..."
    wget -q --show-progress --progress=bar:force:noscroll -P ./data/genome/ http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
    pigz -cd -p $THREADS ./data/genome/hg38.fa.gz > ./data/genome/hg38.fa
    bwa index ./data/genome/hg38.fa
    rm -rf ./data/genome/hg38.fa.gz
else
    echo "Humane genome 38 found. Assuming it is bwa indexed. Skipping downlaoding, unzipping and indexing it."
fi


if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_GRCh38_Ensemblv99.oneline_seqs.fa ]]; then 
    echo "Downloading and unzipping homo sapiens genome..."
    wget -q --show-progress --progress=bar:force:noscroll -P ./data/ http://itmat.data-simulators.s3.amazonaws.com/BEERS2/CAMPAREE_RESOURCE_FILES/HomoSapiens_GRCh38_Ensemblv99__Resource_files.tar.gz
    pigz -cd -p $THREADS ./data/HomoSapiens_GRCh38_Ensemblv99__Resource_files.tar.gz | tar xf - -C ./data/
    rm -rf ./data/HomoSapiens_GRCh38_Ensemblv99__Resource_files.tar.gz
else
    echo "Homo sapiens genome exists. Skipping downloading it."
fi
if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/star_index.genome/Genome ]]; then 
    echo "Downloading and unzipping star index genome..."
    wget -q --show-progress --progress=bar:force:noscroll -P ./data/ http://itmat.data-simulators.s3.amazonaws.com/BEERS2/CAMPAREE_RESOURCE_FILES/HomoSapiens_GRCh38_Ensemblv99__STAR_index.tar.gz
    pigz -cd -p $THREADS ./data/HomoSapiens_GRCh38_Ensemblv99__STAR_index.tar.gz | tar xf - -C ./data/
    rm -rf ./data/HomoSapiens_GRCh38_Ensemblv99__STAR_index.tar.gz
else
    echo "Start index genome already exists. Skipping downloading it."
fi 

Both FASTQ files already exist in the data folder. Skipping fasterq-dump.
Trimmed files by trim_galore already exist. Skipping trim_galore.
Humane genome 38 found. Assuming it is bwa indexed. Skipping downlaoding, unzipping and indexing it.
Homo sapiens genome exists. Skipping downloading it.
Start index genome already exists. Skipping downloading it.


We then skip the step of running the generate_L1EM_fasta_and_index.sh by running the commands to adjust for proper paths and skip it if it is already done.

In [6]:
%%bash
if [[ ! -f ./L1EM/annotation/L1EM.400.fa ]]; then 
    echo "Creating L1EM.400.fa from L1EM.400.bed..."
    bedtools getfasta -s -name -fi ./data/genome/hg38.fa -bed ./L1EM/annotation/L1EM.400.bed > ./L1EM/annotation/L1EM.400.fa
else
    echo "L1EM.400.fa already exists. Skipping its creation."
fi
if [[ ! -f ./L1EM/annotation/L1EM.400.fa.sa ]]; then 
    echo "Indexing the L1EM.400.fa..."
    bwa index ./L1EM/annotation/L1EM.400.fa
else
    echo "L1EM.400.fa was already indexed. Skipping its indexing."
fi


L1EM.400.fa already exists. Skipping its creation.
L1EM.400.fa was already indexed. Skipping its indexing.


We then have to generate the appropriate annotation files. However, as pygtftk is not updated and has conflicting requirements with the rest of the project we are going to be providing the resulting file from runnning the command:

In [ ]:
#!gtftk bed_to_gtf -i ./L1EM/annotation/L1EM.400.bed -o ./annotations/original.gtf

We then continue to do various edits that are needed for the various runs that we are going to do. We also have to remove the colons (:) as Camparee is not compatible with them althouth they are not against the gtf standard.

In [40]:
%%bash
# Make the colons into underscores
if [[ ! -f ./annotations/nocolon.gtf ]]; then 
    sed -E 's/(\.chr([1-9]|1[0-9]|2[0-2]|x|y|mt|un)):/\1_/gI' ./annotations/original.gtf > ./annotations/nocolon.gtf
fi
# Prepare for the camparee format
if [[ ! -f ./annotations/exons.gtf ]]; then 
    sed 's/\ttranscript\t/\texon\t/g' ./annotations/nocolon.gtf > ./annotations/exons.gtf
fi
mkdir -p ./annotations/camparee
# if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz ]]; then 
#     wget -q --show-progress --progress=bar:force:noscroll -P ./data/HomoSapiens_GRCh38_Ensemblv99/ ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
# fi
# if [[ ! -f ./data/HomoSapiens_GRCh38_Ensemblv99/Homo_sapiens.GRCh38.99.gtf.gz ]]; then 
#     wget -q --show-progress --progress=bar:force:noscroll -P ./data/HomoSapiens_GRCh38_Ensemblv99/ ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz
# fi
if [[ ! -f ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.txt ]]; then 
    $CONDA_PREFIX/lib/python3.11/site-packages/camparee/bin/format_reference_files_for_camparee.py -n HomoSapiens_hg38 -o ./annotations/camparee/ -g ./data/genome/hg38.fa -a ./annotations/exons.gtf -p
fi
if [[ ! -f ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.chrNamesfix.txt ]]; then 
    # sed -E 's/^(chr)([1-9]|1[0-9]|2[0-2]|x|y|mt|un)(\t)/\3_/' ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.txt > ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.chrNamesfix.txt
    sed -E 's/^(chr)([^[:space:]]+)/\2/' ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.txt > ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.chrNamesfix.txt
fi
if [[ ! -f ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.sorted.txt ]]; then 
    sort -k1,1 -k4,4n -k5,5n ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.chrNamesfix.txt > ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.sorted.txt
fi
if [[ ! -f ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.ashes.txt ]]; then 
    grep -E '^#|L[^ \t]*\.1\.' ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.sorted.txt > ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.ashes.txt
fi

cp ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.ashes.txt ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_hg38.annotation.ashes.txt
cp ./annotations/camparee/HomoSapiens_hg38/HomoSapiens_hg38.annotation.sorted.txt ./data/HomoSapiens_GRCh38_Ensemblv99/HomoSapiens_hg38.annotation.mixed.txt

We will try to run the flux-simulator now.

To do so we firstly need to create a temporary file for it, split our fasta file into different transcript files and run it.

In [ ]:
%%bash
mkdir -p /tmp/fluxsim
flux-simulator -p ./data/flux/flux.par -x -l -s


Then we want to do 2 different runs of camparee (following the paper) to simualte the runs.

In [48]:
%%bash
camparee -c ./configs/camparee.ashes.yaml -r1
# camparee -c ./configs/camparee.mixed.yaml -r2

Running CAMPAREE using the serial job scheduler.
With the following default scheduler parameters:
 	-default_submission_args : None
	-default_memory_in_mb : 26000
	-default_num_processors : 10


And a maximum job resubmission limit of 3.
Execution of the Expression Pipeline Started...
Running jobs:0 | Pending jobs:5 | Resub jobs:0 | Completed jobs:0
--Check 5 pending jobs for satisfied dependencies:
	Submitting GenomeAlignmentStep command to serial for sample SRR3997504.
	Finished submitting GenomeAlignmentStep command to serial for sample SRR3997504.
Running jobs:0 | Pending jobs:4 | Resub jobs:0 | Completed jobs:1
--Check 4 pending jobs for satisfied dependencies:
	Submitting GenomeBamIndexStep command to serial for sample SRR3997504.
	Finished submitting GenomeBamIndexStep command to serial for sample SRR3997504.
Running jobs:0 | Pending jobs:3 | Resub jobs:0 | Completed jobs:2
--Check 3 pending jobs for satisfied dependencies:
	Submitting VariantsFinderStep command to serial for sample SRR3997504.
	Finished submitting VariantsFinderStep command to serial for sample SRR3997504.
	Submitting IntronQuantificationStep command to serial for sample SRR3997504.
	Finished submittin

We then use beer2 to account for the errors

In [ ]:
# %%bash
# %conda install -y numpy=1.25.2
%pip install git+https://github.com/itmat/BEERS2
!beers --configfile ./configs/beers2.ashes.yaml --jobs 4
# beers --configfile ./configs/beers2.mixed.yaml --jobs 4

TypeError in file /opt/anaconda3/envs/L1EM/lib/python3.11/site-packages/beers/Snakefile, line 83:
C function scipy.spatial._qhull._barycentric_coordinates has wrong signature (expected void (int, double *, double *, double *), got void (int, double *, double const *, double *))
  File "/opt/anaconda3/envs/L1EM/lib/python3.11/site-packages/beers/Snakefile", line 83, in <module>
  File "/opt/anaconda3/envs/L1EM/lib/python3.11/site-packages/beers/library_prep/library_prep_pipeline.py", line 59, in validate
  File "/opt/anaconda3/envs/L1EM/lib/python3.11/importlib/__init__.py", line 126, in import_module
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>

We then take the output of the beers and we merge

In [ ]:
%%bash
